In [ ]:
# Loading drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

2.4.0


In [ ]:
# All general imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer 

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Embedding, Reshape, Conv2D, MaxPool2D, Concatenate, Flatten, Dropout, Dense, Bidirectional, GlobalAveragePooling1D, GRU, GlobalMaxPooling1D, concatenate
from keras.optimizers import Adam
from keras.layers import LSTM, GRU, Conv1D, MaxPool1D, Activation, Add

from keras.models import Model, Sequential
from keras.layers.core import SpatialDropout1D

from keras.engine.topology import Layer
from keras.layers import Dense, Input, Embedding, Dropout, Activation, Conv1D, Softmax
from keras import initializers, regularizers, constraints, optimizers, layers
from keras import backend as K

from keras.callbacks import EarlyStopping

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
import io, os, gc

In [ ]:
# Setting the working directory 
!ls
%cd drive/My\ Drive/Fake_News_Data
!pwd

drive  sample_data
/content/drive/My Drive/Fake_News_Data
/content/drive/My Drive/Fake_News_Data


In [ ]:
#################### Importing FNID Datasets ####################
# Train set
train_df = pd.read_csv('fnn_train.csv')
print(train_df.columns)
train_df.head()
le = LabelEncoder()
train_df['label_fnn'] = le.fit_transform(train_df['label_fnn'])
train_df.head()

# dev set
dev_df = pd.read_csv('fnn_dev.csv')
print(dev_df.columns)
dev_df['label_fnn'] = le.transform(dev_df['label_fnn'])
dev_df.head()

# Test set
test_df = pd.read_csv('fnn_test.csv')
print(test_df.columns)
test_df['label_fnn'] = le.transform(test_df['label_fnn'])
test_df.head()

Index(['id', 'date', 'speaker', 'statement', 'sources',
       'paragraph_based_content', 'fullText_based_content', 'label_fnn',
       'best_novelty', 'best_emotion'],
      dtype='object')
Index(['id', 'date', 'speaker', 'statement', 'sources',
       'paragraph_based_content', 'fullText_based_content', 'label_fnn',
       'best_novelty', 'best_emotion'],
      dtype='object')
Index(['id', 'date', 'speaker', 'statement', 'sources',
       'paragraph_based_content', 'fullText_based_content', 'label_fnn',
       'best_novelty', 'best_emotion'],
      dtype='object')


,id,date,speaker,statement,sources,paragraph_based_content,fullText_based_content,label_fnn,best_novelty,best_emotion
0,1678,2010-04-11T16:37:40-04:00,Jon Kyl,"""President Obama himself attempted to filibust...",['http://abcnews.go.com/ThisWeek/video/supreme...,['U.S. Supreme Court Justice John Paul Stevens...,U.S. Supreme Court Justice John Paul Stevens a...,1,0,0
1,1820,2010-05-23T18:11:09-04:00,Michael Steele,"In Hawaii, ""they don't have a history of throw...",['http://www.starbulletin.com/news/bulletin/94...,"[""On ABC's This Week, the chairmen of the Repu...","On ABC's This Week, the chairmen of the Republ...",1,1,1
2,1624,2010-03-26T10:24:21-04:00,John Boehner,"""Our national debt ... is on track to exceed t...",['http://www.desmoinesregister.com/article/201...,['Ever since Barack Obama became president and...,Ever since Barack Obama became president and b...,1,1,0
3,1576,2010-03-12T11:45:14-05:00,America's Health Insurance Plans,"""Health insurance companies' costs are only 4 ...","['http://www.youtube.com/watch?v=4O8CxZ1OD58',...","[""As the battle over health care reform approa...",As the battle over health care reform approach...,1,1,0
4,1770,2010-05-07T11:54:44-04:00,Michael Bloomberg,"""We can prevent terror suspects from boarding ...",['http://www.huffingtonpost.com/michael-bloomb...,['In the wake of a foiled car bomb attempt in ...,In the wake of a foiled car bomb attempt in Ti...,1,1,1


In [ ]:
train_lst_1 = train_df['statement'].tolist()
train_lst_2 = train_df['paragraph_based_content']
uq_tr_1 = list(set(train_lst_1))
uq_tr_2 = list(set(train_lst_2))
print(len(uq_tr_1))
train_merged = uq_tr_1 + uq_tr_2
print('Train Length is', len(train_merged))
test_lst_1 = test_df['statement'].tolist()
test_lst_2 = test_df['paragraph_based_content']
uq_ts_1 = list(set(test_lst_1))
uq_ts_2 = list(set(test_lst_2))
test_merged = uq_ts_1 + uq_ts_2
dev_lst_1 = dev_df['statement'].tolist()
dev_lst_2 = dev_df['paragraph_based_content']
uq_dv_1 = list(set(dev_lst_1))
uq_dv_2 = list(set(dev_lst_2))
dev_merged = uq_dv_1 + uq_dv_2
total_dataset = train_merged + dev_merged + test_merged
print('Dataset length is', len(total_dataset))

15204
Train Length is 30415
Dataset length is 34638


In [ ]:
# Defining the tokenizer
def get_tokenizer(vocabulary_size):
  print('Training tokenizer...')
  tokenizer = Tokenizer(num_words= vocabulary_size)
  tweet_text = []
  print('Read {} Sentences'.format(len(total_dataset)))
  tokenizer.fit_on_texts(total_dataset)
  return tokenizer

In [ ]:
# For getting the embedding matrix
def get_embeddings():
  print('Generating embeddings matrix...')
  embeddings_file = 'glove.6B.300d.txt'
  embeddings_index = dict()
  with open(embeddings_file, 'r', encoding="utf-8") as infile:
    for line in infile:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      embeddings_index[word] = vector
	# create a weight matrix for words in training docs
  vocabulary_size = len(embeddings_index)
  embeddinds_size = list(embeddings_index.values())[0].shape[0]
  print('Vocabulary = {}, embeddings = {}'.format(vocabulary_size, embeddinds_size))
  tokenizer = get_tokenizer(vocabulary_size)
  embedding_matrix = np.zeros((vocabulary_size, embeddinds_size))
  considered = 0
  total = len(tokenizer.word_index.items())
  for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
      print(word, index)
      continue
    else:
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
        considered += 1
  print('Considered ', considered, 'Left ', total - considered)			
  return embedding_matrix, tokenizer, embeddings_index

In [ ]:
def get_data(tokenizer, MAX_LENGTH, input_df):
  print('Loading data')
  X1, X2, Y = [], [], []
	# with open(input_file) as infile:
	# 	for line in infile:
	# 		data = line.split(',')
	# 		text, annotation = data[2], data[1]
			
	# 		if annotation == "MET":
	# 			X.append(text)
	# 			Y.append("1")
	# 		elif annotation == "Non_MET" or annotation == "Help":	
	# 			X.append(text)
	# 			Y.append("0")
  X1 = input_df['paragraph_based_content'].tolist()
  X2 = input_df['statement'].tolist()
  Y = input_df['label_fnn'].tolist()
  
  assert len(X1) == len(X2) == len(Y)
  sequences_1 = tokenizer.texts_to_sequences(X1)
  sequences_2 = tokenizer.texts_to_sequences(X2)
	# for i, s in enumerate(sequences):
	# 	sequences[i] = sequences[i][-250:]
  X1 = pad_sequences(sequences_1, maxlen=MAX_LENGTH)
  X2 = pad_sequences(sequences_2, maxlen=MAX_LENGTH)
  Y_fnd = np.array(Y)

  return X1, X2, Y_fnd

In [ ]:
embedding_matrix, tokenizer, embeddings_index = get_embeddings()

Generating embeddings matrix...
Vocabulary = 400000, embeddings = 300
Training tokenizer...
Read 34638 Sentences
Considered  62687 Left  49697


In [ ]:
MAX_LENGTH = 100
# read ml data
X1, X2, Y_fnd = get_data(tokenizer, MAX_LENGTH, train_df)

Loading data


In [ ]:
X1_val, X2_val, Y_fnd_val = get_data(tokenizer, MAX_LENGTH, dev_df)

Loading data


In [ ]:
X1_test, X2_test, Y_fnd_test = get_data(tokenizer, MAX_LENGTH, test_df)

Loading data


In [ ]:
# Creating one-hot encodings
y_train_fnd = keras.utils.to_categorical(Y_fnd)
print(y_train_fnd)
y_test_fnd = keras.utils.to_categorical(Y_fnd_test)
print(y_test_fnd)
y_val_fnd = keras.utils.to_categorical(Y_fnd_val)
print(y_val_fnd)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 0.]
 [0. 1.]
 [1. 0.]
 ...
 [0. 1.]
 [1. 0.]
 [0. 1.]]


In [ ]:
print("Training Set")
print("-" * 10)
print(f"x1_train: {X1.shape}")
print(f"x2_train: {X2.shape}")
print(f"y_train_cs : {y_train_fnd.shape}")

print("-" * 10)
print(f"x1_val:   {X1_val.shape}")
print(f"x2_val:   {X2_val.shape}")
print(f"y_val_cs :   {y_val_fnd.shape}")
print("-" * 10)
print("Test Set")

Training Set
----------
x1_train: (15212, 100)
x2_train: (15212, 100)
y_train_cs : (15212, 2)
----------
x1_val:   (1058, 100)
x2_val:   (1058, 100)
y_val_cs :   (1058, 2)
----------
Test Set


In [ ]:
NUM_CLASSES = 2

MAX_SEQUENCE_LENGTH = 100

NUM_LSTM_UNITS = 128

MAX_NUM_WORDS = embedding_matrix.shape[0]

NUM_EMBEDDING_DIM = embedding_matrix.shape[1]

In [ ]:
print('Getting Text CNN model...')
filter_sizes = [2, 3, 5]
num_filters = 128	#Hyperparameters 32,64,128; 0.2,0.3,0.4
drop = 0.4
top_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ), 
    dtype='int32')
bm_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ), 
    dtype='int32')

embedding_layer = Embedding(
    MAX_NUM_WORDS, NUM_EMBEDDING_DIM)
top_embedded = embedding_layer(
    top_input)
bm_embedded = embedding_layer(
    bm_input)
reshape = Reshape((MAX_SEQUENCE_LENGTH, NUM_EMBEDDING_DIM, 1))(top_embedded)
reshape_1 = Reshape((MAX_SEQUENCE_LENGTH, NUM_EMBEDDING_DIM, 1))(bm_embedded)
conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], NUM_EMBEDDING_DIM),  padding='valid', kernel_initializer='normal',  activation='relu')(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], NUM_EMBEDDING_DIM),  padding='valid', kernel_initializer='normal',  activation='relu')(reshape_1)
#conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim),  padding='valid', kernel_initializer='normal', activation='relu')(reshape)
maxpool_0 = MaxPool2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPool2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
#maxpool_2 = MaxPool2D(pool_size=(MAX_LENGTH - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)
concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1])
flatten = Flatten()(concatenated_tensor)
dropout = Dropout(drop)(flatten)
predictions = Dense(units=NUM_CLASSES, activation='sigmoid')(dropout)

model = Model(
    inputs=[top_input, bm_input], 
    outputs=predictions)

Getting Text CNN model...


In [ ]:
from keras.optimizers import Adam
lr = 1e-3
opt = Adam(lr=lr, decay=lr/50)
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [ ]:
BATCH_SIZE = 512
NUM_EPOCHS = 50
stop = [EarlyStopping(monitor='val_loss', patience=0.001)]
history = model.fit(x=[X1, X2],
                    y=y_train_fnd,
                    batch_size=BATCH_SIZE,
                    epochs=NUM_EPOCHS,
                    validation_data=(
                      [X1_val, X2_val], 
                      y_val_fnd
                    ),
                    shuffle=True,
                    callbacks=stop,
          )

Epoch 1/50
30/30 [==============================] - 43s 1s/step - loss: 0.6907 - accuracy: 0.5312 - val_loss: 0.6398 - val_accuracy: 0.6796
Epoch 2/50
30/30 [==============================] - 33s 1s/step - loss: 0.5968 - accuracy: 0.7130 - val_loss: 0.5946 - val_accuracy: 0.6909
Epoch 3/50
30/30 [==============================] - 33s 1s/step - loss: 0.4931 - accuracy: 0.7946 - val_loss: 0.5758 - val_accuracy: 0.6957
Epoch 4/50
30/30 [==============================] - 33s 1s/step - loss: 0.3820 - accuracy: 0.8644 - val_loss: 0.5820 - val_accuracy: 0.6871


In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report
predictions = model.predict(
    [X1_test, X2_test])

In [ ]:
y_pred = [idx for idx in np.argmax(predictions, axis=1)]
print('FNID Accuracy is')
print(metrics.accuracy_score(Y_fnd_test, y_pred)*100)
print(classification_report(Y_fnd_test, y_pred, target_names = ['fake', 'real']))

FNID Accuracy is
79.79127134724858
              precision    recall  f1-score   support

        fake       0.67      0.96      0.79       418
        real       0.96      0.69      0.81       636

    accuracy                           0.80      1054
   macro avg       0.82      0.83      0.80      1054
weighted avg       0.85      0.80      0.80      1054



In [ ]:
# from keras import Input
# from keras.layers import Embedding,LSTM, concatenate, Dense
# from keras.models import Model

top_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ), 
    dtype='int32')
bm_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ), 
    dtype='int32')

embedding_layer = Embedding(
    MAX_NUM_WORDS, NUM_EMBEDDING_DIM)
top_embedded = embedding_layer(
    top_input)
bm_embedded = embedding_layer(
    bm_input)

shared_lstm = LSTM(NUM_LSTM_UNITS)
top_output = shared_lstm(top_embedded)
bm_output = shared_lstm(bm_embedded)

merged = concatenate(
    [top_output, bm_output], 
    axis=-1)

dense =  Dense(
    units=NUM_CLASSES, 
    activation='softmax')
predictions = dense(merged)

model = Model(
    inputs=[top_input, bm_input], 
    outputs=predictions)

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 100, 300)     120000000   input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 128)          219648      embedding_2[0][0]          

In [ ]:
from keras.optimizers import Adam
lr = 1e-3
opt = Adam(lr=lr, decay=lr/50)
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [ ]:
BATCH_SIZE = 512
NUM_EPOCHS = 50
stop = [EarlyStopping(monitor='val_loss', patience=0.001)]
history = model.fit(x=[X1, X2],
                    y=y_train_fnd,
                    batch_size=BATCH_SIZE,
                    epochs=NUM_EPOCHS,
                    validation_data=(
                      [X1_val, X2_val], 
                      y_val_fnd
                    ),
                    shuffle=True,
                    callbacks=stop,
          )

Epoch 1/50
30/30 [==============================] - 38s 1s/step - loss: 0.6803 - accuracy: 0.5624 - val_loss: 0.6150 - val_accuracy: 0.6664
Epoch 2/50
30/30 [==============================] - 34s 1s/step - loss: 0.5273 - accuracy: 0.7472 - val_loss: 0.6017 - val_accuracy: 0.6711
Epoch 3/50
30/30 [==============================] - 34s 1s/step - loss: 0.3179 - accuracy: 0.8726 - val_loss: 0.7802 - val_accuracy: 0.6597


In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report
predictions = model.predict(
    [X1_test, X2_test])

In [ ]:
y_pred = [idx for idx in np.argmax(predictions, axis=1)]
print('FNID Accuracy is')
print(metrics.accuracy_score(Y_fnd_test, y_pred)*100)
print(classification_report(Y_fnd_test, y_pred, target_names = ['fake', 'real']))

FNID Accuracy is
74.38330170777988
              precision    recall  f1-score   support

        fake       0.62      0.91      0.74       418
        real       0.91      0.64      0.75       636

    accuracy                           0.74      1054
   macro avg       0.77      0.77      0.74      1054
weighted avg       0.80      0.74      0.74      1054

